# Extract data for urban calculations

Test input for Tanzania

0. Select focal ADM, buffer by 1km, rasterize as [0, 1]
1. Download DEM data from ASTER, mosaick
2. Calculate slope of DEM
3. Extract water layer from Globcover
4. Rasterize building footprints
5. Select population layer
6. Standardize all rasters to population layer  
   a. Set area outside of focal admin to NoData  
   b. Set everything to 16bit  
   
   


In [35]:
import sys, os, importlib, shutil
import requests
import rasterio, elevation, richdem
import rasterio.warp
from rasterio import features

import pandas as pd
import geopandas as gpd
import numpy as np

from shapely.geometry import MultiPolygon, Polygon, box, Point

#Import raster helpers
sys.path.append("../../../gostrocks/src")

import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint

#Import GOST urban functions
sys.path.append("../../")
import src.UrbanRaster as urban
import src.urban_helper as helper



In [2]:
global_bounds = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
global_bounds_adm2 = "/home/public/Data/GLOBAL/ADMIN/Admin2_Polys.shp"

inG = gpd.read_file(global_bounds)
inG2 = gpd.read_file(global_bounds_adm2)

runSmall = True
runLarge = True

In [11]:
importlib.reload(helper)
importlib.reload(rMisc)

def calculate_urban(iso3, inG, inG2, pop_files, ea_file, km=True, small=True):
    global_landcover  = "/home/public/Data/GLOBAL/LANDCOVER/GLOBCOVER/2015/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7.tif"
    global_ghspop = "/home/public/Data/GLOBAL/Population/GHS/250/GHS_POP_E2015_GLOBE_R2019A_54009_250_V1_0.tif"
    global_ghspop_1k = "/home/public/Data/GLOBAL/Population/GHS/GHS_POP_E2015_GLOBE_R2019A_54009_1K_V1_0.tif"
    global_ghbuilt = "/home/public/Data/GLOBAL/URBAN/GHS/GHS_1K_BUILT/GHS_BUILT_LDS2014_GLOBE_R2018A_54009_1K_V1_0.tif"
    global_dem_1k = "/home/public/Data/GLOBAL/ELEV/noaa_1km.tif"
    ghs_smod = "/home/public/Data/GLOBAL/URBAN/GHS/GHS_SMOD/GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V2_0.tif"
    ghsl_vrt = "/home/public/Data/GLOBAL/GHSL/ghsl.vrt"

    output_folder = "/home/wb411133/temp/%s_URBAN_DATA_new_naming" % iso3
    inD = inG.loc[inG['ISO3'] == iso3]
    inD['geometry'] = inD['geometry'].apply(lambda x: x.buffer(500))
    inD = inD.to_crs({'init':'epsg:4326'})
    
    inD2 = inG2.loc[inG2['ISO3'] == iso3]
    inD2 = inD2.to_crs({'init':'epsg:4326'})  
    
    ### Process 1km data
    if km:
        xx = helper.urban_country(iso3, output_folder, inD, pop_files,
                                final_folder="FINAL_STANDARD_1KM", ghspop_suffix="1k")
        adm2_res = os.path.join(xx.final_folder, "URBAN_ADMIN2_STATS_COMPILED.csv") 
        ea_res   = os.path.join(xx.final_folder, "URBAN_COMMUNE_STATS_COMPILED.csv")
        tPrint("***** Extracting Global Layers %s" % iso3)
        xx.extract_layers(global_landcover, global_ghspop, global_ghspop_1k, global_ghbuilt, ghsl_vrt, ghs_smod)
        tPrint("***** Downloading and processing elevation %s" % iso3)
        xx.process_dem(global_dem=global_dem_1k)
        tPrint("***** Standardizing rasters")
        xx.standardize_rasters()
        tPrint("***** Calculating Urban")
        xx.calculate_urban()
        tPrint("***** Evaluating Data")
        xx.evaluateOutput()
    
    ### Process 250m data 
    if small:
        xx = helper.urban_country(iso3, output_folder, inD, pop_files)
        tPrint("***** Extracting Global Layers %s" % iso3)
        xx.extract_layers(global_landcover, global_ghspop, global_ghspop_1k, global_ghbuilt, ghsl_vrt, ghs_smod)
        tPrint("***** Downloading and processing elevation %s" % iso3)
        xx.process_dem(global_dem=global_dem_1k)
        tPrint("***** Standardizing rasters")
        xx.standardize_rasters()
        tPrint("***** Calculating Urban")
        xx.calculate_urban()
        xx.evaluateOutput()
        tPrint("***** Calculating Zonal admin2")
        if os.path.exists(ea_file):
            if not os.path.exists(os.path.join(output_folder, "URBAN_ADMIN2_STATS_COMPILED.csv")):
                zonal_adm2 = xx.pop_zonal_admin(inD2)
                zonal_adm2.to_csv(os.path.join(output_folder, "URBAN_ADMIN2_STATS_COMPILED.csv"))
            tPrint("***** Calculating Zonal communes")
            if not os.path.exists(os.path.join(output_folder, "URBAN_COMMUNE_STATS_COMPILED.csv")):
                inEA = gpd.read_file(ea_file)
                zonal_ea = xx.pop_zonal_admin(inEA)
                zonal_ea.to_csv(os.path.join(output_folder, "URBAN_COMMUNE_STATS_COMPILED.csv"))

In [ ]:
### JUST EXTRACT DATA
iso3 = "TUR"
pop_files = []
output_folder = "/home/wb411133/temp/%s_URBAN_DATA_new_naming" % iso3
calculate_urban(iso3, inG, inG2, pop_files, '', small=runSmall, km=runLarge)

In [ ]:
# Process ETH
iso3 = "ETH"
local_path = "/home/public/Data/COUNTRY/{country}/WORLDPOP/".format(country=iso3)
pop_2015_un = os.path.join(local_path, "%s_ppp_2015_UNadj.tif" % iso3.lower())
pop_2018_un = os.path.join(local_path, "%s_ppp_2016_UNadj.tif" % iso3.lower())
pop_files = [[pop_2015_un, "%s_upo15.tif" % iso3.lower()], 
             [pop_2018_un, "%s_upo16.tif" % iso3.lower()]]
output_folder = "/home/wb411133/temp/%s_URBAN_DATA_new_naming" % iso3
ea_file = ''

calculate_urban(iso3, inG, inG2, pop_files, ea_file, small=runSmall, km=runLarge)

In [ ]:
importlib.reload(helper)
# Process COL
iso3 = "COL"
local_path = "/home/public/Data/COUNTRY/{country}/POPULATION/WORLDPOP/".format(country=iso3)
pop_2015_un = os.path.join(local_path, "%s_ppp_2015_UNadj.tif" % iso3.lower())
pop_files = [[pop_2015_un, "%s_upo15.tif" % iso3.lower()]]
output_folder = "/home/wb411133/temp/%s_URBAN_DATA_new_naming" % iso3
ea_file = ''

calculate_urban(iso3, inG, inG2, pop_files, ea_file, small=runSmall, km=runLarge)

In [ ]:
importlib.reload(helper)
# Process EGY
iso3 = "EGY"
local_path = "/home/public/Data/COUNTRY/{country}/POPULATION/WORLDPOP/".format(country=iso3)
pop_2015_un = os.path.join(local_path, "%s_ppp_2015_UNadj.tif" % iso3.lower())
pop_2018_un = os.path.join(local_path, "%s_ppp_2013_UNadj.tif" % iso3.lower())
pop_files = [[pop_2015_un, "%s_upo15.tif" % iso3.lower()], 
             [pop_2018_un, "%s_upo16.tif" % iso3.lower()]]
output_folder = "/home/wb411133/temp/%s_URBAN_DATA_new_naming" % iso3

ea_file = ''

calculate_urban(iso3, inG, inG2, pop_files, ea_file, small=runSmall, km=runLarge)

In [77]:
importlib.reload(helper)
# Process AGO
iso3 = "AGO"
local_path = "/home/public/Data/COUNTRY/{country}/POPULATION/WORLDPOP/".format(country=iso3)
pop_2015_un = os.path.join(local_path, "%s_ppp_2015_UNadj.tif" % iso3.lower())
pop_2018_un = os.path.join(local_path, "%s_ppp_2018_UNadj.tif" % iso3.lower())
pop_files = [[pop_2015_un, "%s_upo15.tif" % iso3.lower()], 
             [pop_2018_un, "%s_upo18.tif" % iso3.lower()]]
output_folder = "/home/wb411133/temp/%s_URBAN_DATA_new_naming" % iso3
ea_file = os.path.join(output_folder, 'admin', 'bairros.shp')

calculate_urban(iso3, inG, inG2, pop_files, ea_file, small=runSmall, km=runLarge)

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


16:49:53	***** Extracting Global Layers AGO
16:49:53	***** Downloading and processing elevation AGO
16:49:53	***** Standardizing rasters
/home/wb411133/temp/AGO_URBAN_DATA_new_naming/ago_adm.tif
/home/wb411133/temp/AGO_URBAN_DATA_new_naming/ago_gpo.tif
/home/wb411133/temp/AGO_URBAN_DATA_new_naming/ago_wat_lc.tif
/home/wb411133/temp/AGO_URBAN_DATA_new_naming/ago_wat.tif
/home/wb411133/temp/AGO_URBAN_DATA_new_naming/ago_slo.tif
/home/wb411133/temp/AGO_URBAN_DATA_new_naming/ago_ele.tif
/home/wb411133/temp/AGO_URBAN_DATA_new_naming/ago_gsmod.tif
/home/wb411133/temp/AGO_URBAN_DATA_new_naming/ago_gbu.tif
/home/wb411133/temp/AGO_URBAN_DATA_new_naming/ago_upo15.tif
/home/wb411133/temp/AGO_URBAN_DATA_new_naming/ago_upo18.tif
/home/wb411133/temp/AGO_URBAN_DATA_new_naming/ago1k_gpo.tif
16:49:53	***** Calculating Urban
/home/wb411133/temp/AGO_URBAN_DATA_new_naming/FINAL_STANDARD_1KM/ago1k_upo15.tif
/home/wb411133/temp/AGO_URBAN_DATA_new_naming/FINAL_STANDARD_1KM/ago1k_upo18.tif
/home/wb411133/temp

In [24]:
importlib.reload(helper)
# Process GHA
iso3 = "GHA"
local_path = "/home/public/Data/COUNTRY/{country}/POPULATION/WORLDPOP/".format(country=iso3)
pop_2015_un = os.path.join(local_path, "%s_ppp_2015_UNadj.tif" % iso3.lower())
pop_2018_un = os.path.join(local_path, "%s_ppp_2017_UNadj.tif" % iso3.lower())
pop_2015_con = os.path.join(local_path, "ppp_prj_2015_%s_UNadj.tif" % iso3)
pop_2018_con = os.path.join(local_path, "ppp_prj_2017_%s_UNadj.tif" % iso3)

pop_files = [[pop_2015_un, "%s_upo15.tif" % iso3.lower()], 
             [pop_2018_un, "%s_upo17.tif" % iso3.lower()], 
             [pop_2015_con, "%s_cpo15.tif" % iso3.lower()], 
             [pop_2018_con, "%s_cpo17.tif" % iso3.lower()]]
output_folder = "/home/wb411133/temp/%s_URBAN_DATA_new_naming" % iso3
ea_file = os.path.join(output_folder, 'FINAL_EAS.shp')

calculate_urban(iso3, inG, inG2, pop_files, ea_file, small=runSmall, km=runLarge)

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


14:58:30	***** Extracting Global Layers GHA
14:58:30	***** Downloading and processing elevation GHA
14:58:30	***** Standardizing rasters
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/gha_adm.tif
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/gha_gpo.tif
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/gha_wat_lc.tif
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/gha_wat.tif
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/gha_slo.tif
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/gha_ele.tif
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/gha_gsmod.tif
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/gha_gbu.tif
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/gha_upo15.tif
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/gha_upo17.tif
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/gha_cpo15.tif
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/gha_cpo17.tif
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/gha1k_gpo.tif
14:58:30	***** Calculating Urban
/home/wb411133/temp/GHA_URBAN_DATA_new_naming/FINAL_STANDARD_

In [26]:
importlib.reload(helper)
# Process BGD
iso3 = "BGD"
local_path = "/home/public/Data/COUNTRY/{country}/POPULATION/WORLDPOP/".format(country=iso3)
pop_2015_un = os.path.join(local_path, "%s_ppp_2015_UNadj.tif" % iso3.lower())
pop_2018_un = os.path.join(local_path, "%s_ppp_2018_UNadj.tif" % iso3.lower())

pop_files = [[pop_2015_un, "%s_upo15.tif" % iso3.lower()], 
             [pop_2018_un, "%s_upo18.tif" % iso3.lower()]]

output_folder = "/home/wb411133/temp/%s_URBAN_DATA_new_naming" % iso3
ea_file = os.path.join(output_folder, 'mauza11_reprojected.shp')
calculate_urban(iso3, inG, inG2, pop_files, ea_file, small=runSmall, km=runLarge)

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


15:23:57	***** Extracting Global Layers BGD
15:23:57	***** Downloading and processing elevation BGD
15:23:57	***** Standardizing rasters
/home/wb411133/temp/BGD_URBAN_DATA_new_naming/bgd_adm.tif
/home/wb411133/temp/BGD_URBAN_DATA_new_naming/bgd_gpo.tif
/home/wb411133/temp/BGD_URBAN_DATA_new_naming/bgd_wat_lc.tif
/home/wb411133/temp/BGD_URBAN_DATA_new_naming/bgd_wat.tif
/home/wb411133/temp/BGD_URBAN_DATA_new_naming/bgd_slo.tif
/home/wb411133/temp/BGD_URBAN_DATA_new_naming/bgd_ele.tif
/home/wb411133/temp/BGD_URBAN_DATA_new_naming/bgd_gsmod.tif
/home/wb411133/temp/BGD_URBAN_DATA_new_naming/bgd_gbu.tif
/home/wb411133/temp/BGD_URBAN_DATA_new_naming/bgd_upo15.tif
/home/wb411133/temp/BGD_URBAN_DATA_new_naming/bgd_upo18.tif
/home/wb411133/temp/BGD_URBAN_DATA_new_naming/bgd1k_gpo.tif
15:23:57	***** Calculating Urban
/home/wb411133/temp/BGD_URBAN_DATA_new_naming/FINAL_STANDARD_1KM/bgd1k_upo15.tif
/home/wb411133/temp/BGD_URBAN_DATA_new_naming/FINAL_STANDARD_1KM/bgd1k_upo18.tif
/home/wb411133/temp

In [32]:
# Process TZA
iso3 = "TZA"
local_path = "/home/public/Data/COUNTRY/{country}/POPULATION/WORLDPOP/TZA_2015_2018".format(country=iso3)
pop_2015_un = os.path.join(local_path, "%s_ppp_2015_UNadj.tif" % iso3.lower())
pop_2018_un = os.path.join(local_path, "%s_ppp_2018_UNadj.tif" % iso3.lower())
pop_2015_con = os.path.join(local_path, "ppp_prj_2015_%s_UNadj.tif" % iso3)
pop_2018_con = os.path.join(local_path, "ppp_prj_2018_%s_UNadj.tif" % iso3)

pop_files = [[pop_2015_un, "%s_upo15.tif" % iso3.lower()], 
             [pop_2018_un, "%s_upo18.tif" % iso3.lower()], 
             [pop_2015_con, "%s_cpo15.tif" % iso3.lower()], 
             [pop_2018_con, "%s_cpo18.tif" % iso3.lower()]]

output_folder = "/home/wb411133/temp/%s_URBAN_DATA_new_naming" % iso3
ea_file = ''

calculate_urban(iso3, inG, inG2, pop_files, ea_file, small=runSmall, km=runLarge)

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


15:29:46	***** Extracting Global Layers TZA
15:29:46	***** Downloading and processing elevation TZA
15:29:46	***** Standardizing rasters
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/tza_adm.tif
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/tza_gpo.tif
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/tza_wat_lc.tif
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/tza_wat.tif
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/tza_slo.tif
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/tza_ele.tif
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/tza_gsmod.tif
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/tza_gbu.tif
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/tza_upo15.tif
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/tza_upo18.tif
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/tza_cpo15.tif
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/tza_cpo18.tif
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/tza1k_gpo.tif
15:29:46	***** Calculating Urban
/home/wb411133/temp/TZA_URBAN_DATA_new_naming/FINAL_STANDARD_

In [60]:
# Process point location analysis
input_file = os.path.join(output_folder, "sample_imp.csv")
inD = pd.read_csv(input_file)
inD.head()

,fid,interview__key,interview__id,REGION,DISTRICT,WARDCODE,WARDNAME,VILLCODE,VILLNAME,EA,...,hhlocation__Altitude,hhlocation__Timestamp,gps_accu,hhid,missGPS,outlier3la,outlier3lo,gps_imp_la,gps_imp_lo,rvalue_raw_1
0,1,00-00-48-06,38e866fb1dbe4aa185c7cbe577836272,Songwe,9,31,Ivuna,4,Mkomba,8,...,865.000000,2018-04-03T11:18:44,50.0,4806,0,NaN,NaN,-8.506450,32.56220,11.0
1,2,00-02-66-35,15a31f27141b41f18364a3622e14cb16,Rukwa,2,152,Laela,14,Chunya,2,...,1615.979053,2018-09-06T15:34:14,10.0,26635,0,NaN,NaN,-8.516530,32.03525,13.0
2,3,00-07-34-88,6b94172ae591440bac756cf5db7b2b62,Dar Es Salaa,2,92,Kipawa,1,Mogo,46,...,NaN,##N/A##,NaN,73488,1,NaN,NaN,-6.866129,39.19664,30.0
3,4,00-07-69-60,40fdb9ceb04d4c5489837eb699ffd2df,Dodoma,5,311,Ntyuka,1,Ntyuka,11,...,1209.000000,2018-03-17T14:19:56,6.0,76960,0,NaN,NaN,-6.225760,35.75982,13.0
4,5,00-08-94-09,c9ca242c0f0749298350ff2aafd6d066,Dodoma,4,291,Loje,1,Loje,4,...,687.000000,2018-04-03T09:32:51,6.0,89409,0,NaN,NaN,-6.910590,35.98455,11.0


In [61]:
geoms = [Point(row['gps_imp_lo'], row['gps_imp_la']) for idx, row in inD.iterrows()]
inD = gpd.GeoDataFrame(inD, geometry=geoms, crs={'init':'epsg:4326'})

In [68]:
pop_tiffs = ['/home/wb411133/temp/TZA_URBAN_DATA_new_naming/FINAL_STANDARD/tza_upo15.tif',
'/home/wb411133/temp/TZA_URBAN_DATA_new_naming/FINAL_STANDARD/tza_upo18.tif',
'/home/wb411133/temp/TZA_URBAN_DATA_new_naming/FINAL_STANDARD/tza_cpo15.tif',
'/home/wb411133/temp/TZA_URBAN_DATA_new_naming/FINAL_STANDARD/tza_cpo18.tif',
'/home/wb411133/temp/TZA_URBAN_DATA_new_naming/FINAL_STANDARD/tza_gpo.tif']

for pFile in pop_tiffs:
    urb_file = pFile.replace(".tif", "_urban.tif")
    hd_file = pFile.replace(".tif", "_urban_hd.tif")
    for curFile in [urb_file, hd_file]:
        inUrb = rasterio.open(curFile)
        if inD.crs!= inUrb.crs:
            inD = inD.to_crs(inUrb.crs)
        geoms = [(row['geometry'].x, row['geometry'].y) for idx, row in inD.iterrows()]
        urb_res = inUrb.sample(geoms)
        inD[os.path.basename(curFile).replace(".tif","")] = [x[0] for x in list(urb_res)]

In [72]:
pd.DataFrame(inD.drop(['geometry'], axis=1)).to_csv(input_file.replace(".csv", "_urban.csv"))

In [73]:
# Process VNM
iso3 = "VNM"
local_path = "/home/public/Data/COUNTRY/{country}/POPULATION/WORLDPOP/".format(country=iso3)
pop_2015_un = os.path.join(local_path, "%s_ppp_2015_UNadj.tif" % iso3.lower())
pop_2018_un = os.path.join(local_path, "%s_ppp_2018_UNadj.tif" % iso3.lower())
pop_files = [[pop_2015_un, "%s_upo15.tif" % iso3.lower()], 
             [pop_2018_un, "%s_upo18.tif" % iso3.lower()]]
output_folder = "/home/wb411133/temp/%s_URBAN_DATA_new_naming" % iso3
ea_file = os.path.join(output_folder, 'VN_communes2008.shp')

calculate_urban(iso3, inG, inG2, pop_files, ea_file, small=runSmall, km=runLarge)

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


16:34:08	***** Extracting Global Layers VNM
16:34:08	***** Downloading and processing elevation VNM
16:34:08	***** Standardizing rasters
/home/wb411133/temp/VNM_URBAN_DATA_new_naming/vnm_adm.tif
/home/wb411133/temp/VNM_URBAN_DATA_new_naming/vnm_gpo.tif
/home/wb411133/temp/VNM_URBAN_DATA_new_naming/vnm_wat_lc.tif
/home/wb411133/temp/VNM_URBAN_DATA_new_naming/vnm_wat.tif
/home/wb411133/temp/VNM_URBAN_DATA_new_naming/vnm_slo.tif
/home/wb411133/temp/VNM_URBAN_DATA_new_naming/vnm_ele.tif
/home/wb411133/temp/VNM_URBAN_DATA_new_naming/vnm_gsmod.tif
/home/wb411133/temp/VNM_URBAN_DATA_new_naming/vnm_gbu.tif
/home/wb411133/temp/VNM_URBAN_DATA_new_naming/vnm_upo15.tif
/home/wb411133/temp/VNM_URBAN_DATA_new_naming/vnm_upo18.tif
/home/wb411133/temp/VNM_URBAN_DATA_new_naming/vnm1k_gpo.tif
16:34:08	***** Calculating Urban
/home/wb411133/temp/VNM_URBAN_DATA_new_naming/FINAL_STANDARD_1KM/vnm1k_upo15.tif
/home/wb411133/temp/VNM_URBAN_DATA_new_naming/FINAL_STANDARD_1KM/vnm1k_upo18.tif
/home/wb411133/temp

# Debugging

In [ ]:
import json
from rasterio import features
from rasterio.mask import mask

# Extract GHSL without vrt
iso3 = "ETH"
ghsl_vrt = "/home/public/Data/GLOBAL/GHSL/ghsl.vrt"
inD = inG.loc[inG['ISO3'] == iso3]
inR = rasterio.open(ghsl_vrt)
    
    

In [ ]:
bounds = box(*inD.total_bounds)
def getFeatures(gdf):
    #Function to parse features from GeoDataFrame in such a manner that rasterio wants them
    return [json.loads(gdf.to_json())['features'][0]['geometry']]
tD = gpd.GeoDataFrame([[1]], geometry=[bounds])
coords = getFeatures(tD)


In [ ]:
out_img, out_transform = mask(inR, shapes=coords, crop=True)

In [ ]:
in_file = "/home/wb411133/temp/ETH_URBAN_DATA_new_naming/eth_wat_lc.tif"
xx = rasterio.open(in_file)
xx.meta


In [ ]:
data.shape

In [ ]:
b = inD.total_bounds
rasterio.transform.from_bounds(b[0], b[1], b[2], b[3], data.shape[1], data.shape[0])